# Smarter.Camera Documentation

#### Requirements
1. Chrome is the only supported/tested browser, so use Chrome.
2. US-EAST-1 (Virginia) is the region you should work in.
3. If reading this doc directly on Github (as opposed to running the notebook) you'll need to left click hyperlinks and open in a new tab.

## Account Setup

### Sign Up

Create a new account [**here**](https://auth-us-west-2-aws.smarter.camera/signup?response_type=code&client_id=1uc61n2nemgv8v5mlfbsmj7v5g&state=Cognito-eca57bca-666f-0271-7646-d2d05bded099&redirect_uri=https://smarter.camera/html/login-redirect.html).

Verify your email by entering the one time passcode sent to your email.

Next login with your username/password and you'll be redirected to the Settings page in the smarter.camera web app where you can continue settin up your account.


### Set AWS Account ID

After logging into the smarter.camera web app, navigate to the [**Settings**](https://smarter.camera/#!/settings) page.

This is where you need to save your AWS Account ID. The current version Smarter.Camera operates on a Bring Your Own Account (BYOA) model, which means all video, face indexing, training, etc. happens inside your account. Smarter.Camera needs to know your account number so it can assume a role in when it interacts with your data. 

### Enable Cross Account Access

Smarter.Camera doesn't store any of your data, all your data is inside your own AWS Account. Smarter.Camera is just a [Single Page App (SPA)](https://en.wikipedia.org/wiki/Single-page_application) that allows you to easily interact with native AWS services insite your account. The web application needs to assume a role that allows access to your data. A series of IAM roles need to be installed into your account to allow the Smarter.Camera application to function. 

On the [Settings](https://smarter.camera/#!/settings) page, click the button that says "Install Smarter.Camera CF Template". This directs you to the CloudFormation page with parameters preset to your user specific values.

The CloudFormation template used can be viewed [here](
https://s3.amazonaws.com/s3-us-east-1-aws.smarter.camera/app/smater-camera-cf-template.yaml). The parameters are dynamically passed to CloudFormation so they will be prepoulated for you. Notice the SmarterCameraUserId is populated with your specific user id in the Smarter.Camera system. Use all the prepopulated values.

#### CloudFormation Example

![Create Stack Example](./images/createStackExample.png)


#### IAM Roles Created

Role Name|Description|Principal
---|---|---
SmarterCameraProviderRole|This is the role that is used from your browser session in the smarter.camera web application. After you log into smarter.camera, a call is made to [STS](https://docs.aws.amazon.com/STS/latest/APIReference/Welcome.html) to assume the role in your account, which is only authorized for your specific Cognito user.| Your Smarter.Camera User
SmarterCameraDeepLensLambdaRole      | This role is used by the Lambda function you deploy to your DeepLens device. It has permission to talk to IOT/Greengrass for messaging purposes, S3 for face image uploads, Rekognition for face indexing, Kinesis Video for video indexing. | Lambda
SmarterCameraSageMakerExecutionRole|This role is used by [SageMaker](https://aws.amazon.com/sagemaker/) to run [Face Clustering](https://www.pyimagesearch.com/2018/07/09/face-clustering-with-python/) on faces detected by your DeepLens.|SageMaker

### Install DeepLens Lambda Function

After executing the CloudFormation template in the previous step you can log back into Smarter.Camera and you should be directed to the cameras page. At this point the web application has access to your account, but you still need to deploy the Smarter.Camera DeepLens Lambda project to your DeepLens device. You can check out the [source code here](https://github.com/paulfryer/DeepLens/blob/master/StreamFaces/lambda.py).

First deploy the Lambda Function that will be used to send video to Kinesis Video and index faces in Rekognition and S3. 
[**Click Here to deploy**](https://serverlessrepo.aws.amazon.com/applications/arn:aws:serverlessrepo:us-east-1:072676109536:applications~SmarterCamera-DeepLens-PeopleTracking) the function from the [Serverless Repo](https://aws.amazon.com/serverless/serverlessrepo/).

![Serverless Repo](./images/serverlessRepo.png)

##### Change the Lambda IAM Role
Next change the IAM role used by the Lambda function to the role named: **SmarterCameraDeepLensLambdaRole**. That role was installed with the cloud formation template you installed. 

![Change Role](./images/changeRole.png)

##### Publish New Version of the Lambda Function
DeepLens projects only work with [versioned Lambda functions](https://docs.aws.amazon.com/lambda/latest/dg/versioning-intro.html) so you need to publish V1 of your function.

![Publish New Version](./images/publishNewVersion.png)

### Register Your DeepLens Device

Follow instructions for [Registering Your AWS DeepLens Device](https://docs.aws.amazon.com/deeplens/latest/dg/deeplens-getting-started-register.html).

Copy your DeepLens ID. You can find this by viewing the Device properties in the DeepLens console. The value will look like **deeplens_q66CW75eQCicnXQKApC8OA**

![DeepLensID](./images/deepLensID.png)

This ID is used as a key that ties together the DeepLens device, Rekognition Face Collection, S3 Bucket, and SageMaker training jobs.

#### Register your DeepLens with Smarter.Camera

Log into Smarter.Camera and navigate to the Cameras page, here you need to paste the DeepLens device ID and **Register the DeepLens device**. In your account, this will create an S3 bucket, Rekogntion Face Collection, and Kinesis Video Stream with the device ID. You are now ready to start capturing video and photos from your device.

![Register Device](./images/registerDevice.png)


### Create a Smarter.Camera Project & Deploy to DeepLens
Create a new DeepLens Project using the **Face Detection** project template.

![Face Detection Project](./images/faceDetectionProject.png)

Click Next, name your project **Smarter-Camera**. Save the project. 

From the DeepLens project page, Edit the **Smarter-Camera** project.

Add the Lambda Function you previously installed from the Serverless Repo. Click the **Add Function** button. Search for your function by keyword **smart**.

![Find Your Function](./images/findYourFunction.png)

Remove the default Lambda function named **deeplense-face-detection**.

![Remove Default](./images/removeDefault.png)

Your project is now ready to be deployed. Deploy to your Device.

Once the project is deployed it will start recording video to your Kinesis Video stream when faces are detected for 15 seconds.
It will also send faces to the S3 bucket named with your DeviceID, as well as send faces to Rekognition for indexing.

After you capture some video you can try searching for a face.

## Using the Smarter.Camera Application

### Finding People in Video

After you register your device with Smarter.Camera you'll be able to search for faces. Click on the device ID of interest from the Cameras page. This opens a page specific to that camera. You can now upload a photo of a person you want to search for. 

![Search for Face](./images/searchForFace.png)

If any matches are found they will be presented in the chart at the bottom of the page. Each bar in the chart represents a timestamp when the face match was observed. The height of the bar indicates the confidence level that match is the same face you submitted in your query.

Clicking on a timestamp (bar) in the chart will load a video snippet at that time. A time window is based on the **Seconds** field. You can widen or narrow the video window with that setting.

### Clustering Faces

Searching for faces works great if you have pictures of people you want to find. But what if you don't know who has been observed in your video? This is where Face Clustering comes in, you can create a unique set of faces that are observed in your video without even knowing who they are.

From the Cameras page click the **Faces** link to review training jobs and/or launch a new job. Click the **Cluster Faces** button to launch a new training job. This will submit all the faces in your DeepLens S3 bucket to a [custom](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms.html) Smarter.Camera Face Clustering algorithm that runs a SageMaker training job. The out put of this job is a set of folders that contains clustered faces.

![Launch Training](./images/launchTraining.png)

#### Viewing the Output

Once a training job is complete you can download the output by clicking the **Download Output** button. This is a ziped and tarred file that contains faces clustered into folders. **-1** folder contains images that couldn't be clustered.

![Faces](./images/faces.png)

Now that you have the observed faces you can return the face search page to find all the times the face was observed and corresponding video snippets.